参考：https://github.com/liuwei1206/deep-learning/blob/master/AlexNet/main.py

https://blog.csdn.net/xuan_liu123/article/details/89105997

再MNIST 数据集上测试

In [5]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F

from torch.utils.data import DataLoader

import time

In [4]:
# 定义网络结构
class AlexNet(nn.Module):
    
    def __init__(self):
        super(AlexNet, self).__init__()
        # 由于MNIST为28x28， 而最初AlexNet的输入图片是227x227的。所以网络层数和参数需要调节
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),  # inplace = True  为了省空间
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
            )
        self.fc = nn.Sequential(
            nn.Linear(256*3*3, 1024),
            nn.Linear(1024,512),
            nn.Linear(512,10)
            )
        
    def forward(self, x):
        x = self.features(x)
        x = self.fc(x.view(x.size(0), -1))  # x = x.view(-1, 256 * 3 * 3)
        return x

In [6]:
# 数据增强
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),# 对图片进行概率为0.5随机反转
    transforms.RandomGrayscale(),
    transforms.ToTensor()
])

transform1 = transforms.Compose([
    transforms.ToTensor()
]) 

# 加载数据
trainset = torchvision.datasets.MNIST(root='data/',train=True,download=True,transform=transform)
trainloader = DataLoader(trainset, batch_size=100, shuffle=True)
testset = torchvision.datasets.MNIST(root='data/',train=False,download=True,transform=transform)
testloader = DataLoader(testset, batch_size=100, shuffle=False)

In [14]:
model = AlexNet()

criterian = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

#device : GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model.to(device)

print("Start Training!")

num_epoches = 20

for epoch in range(num_epoches):
    running_loss = 0.
    running_acc = 0.
    batch_size = 100
    start = time.time()
    
    for (img, label) in trainloader:
        img = img.to(device)
        label = label.to(device)
        
        outputs = model(img)
        loss = criterian(outputs, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, pred = outputs.max(1)
        num_correct = (pred == label).sum().item()
        running_acc += num_correct
    
    running_loss /= len(trainset)
    running_acc /= len(trainset)
    print("[%d/%d] Loss: %.5f, Acc: %.2f, Time: %.1f s" %(epoch+1, num_epoches, running_loss, 100*running_acc, time.time()-start))

print("Finished Traning")

Start Training!
[1/20] Loss: 0.02299, Acc: 11.94, Time: 15.9 s
[2/20] Loss: 0.02285, Acc: 18.85, Time: 16.0 s
[3/20] Loss: 0.01496, Acc: 53.52, Time: 16.1 s
[4/20] Loss: 0.00544, Acc: 82.31, Time: 15.9 s
[5/20] Loss: 0.00362, Acc: 88.14, Time: 15.9 s
[6/20] Loss: 0.00271, Acc: 91.22, Time: 16.0 s
[7/20] Loss: 0.00218, Acc: 93.03, Time: 16.0 s
[8/20] Loss: 0.00186, Acc: 94.09, Time: 15.9 s
[9/20] Loss: 0.00167, Acc: 94.72, Time: 15.9 s
[10/20] Loss: 0.00150, Acc: 95.27, Time: 15.9 s
[11/20] Loss: 0.00139, Acc: 95.55, Time: 16.1 s
[12/20] Loss: 0.00125, Acc: 95.97, Time: 15.9 s
[13/20] Loss: 0.00117, Acc: 96.28, Time: 15.9 s
[14/20] Loss: 0.00112, Acc: 96.43, Time: 15.8 s
[15/20] Loss: 0.00105, Acc: 96.59, Time: 16.1 s
[16/20] Loss: 0.00100, Acc: 96.77, Time: 15.9 s
[17/20] Loss: 0.00095, Acc: 96.99, Time: 16.0 s
[18/20] Loss: 0.00093, Acc: 97.06, Time: 16.0 s
[19/20] Loss: 0.00089, Acc: 97.15, Time: 15.9 s
[20/20] Loss: 0.00083, Acc: 97.39, Time: 16.1 s
Finished Traning


In [16]:
# 保存模型
torch.save(model, 'model/AlexNet_MNIST.pkl')
net = torch.load('model/AlexNet_MNIST.pkl')

D:\software\Anaconda3\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type AlexNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [18]:
# evaluate
testloss = 0.
testacc = 0.
for (img, label) in testloader:
    img = img.to(device)
    label = label.to(device)
    
    outputs = net(img)
    loss = criterian(outputs, label)
    
    testloss += loss.item()
    _, pred = outputs.max(1)
    num_correct = (pred == label).sum().item()
    testacc += num_correct

testloss /= len(testset)
testacc /= len(testset)
print("Test:Loss:%.5f, Acc:%.2f %%" %(testloss, 100*testacc))

Test:Loss:0.00087, Acc:97.09 %
